In [1]:
import dipy

In [2]:
from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs

In [3]:
from fenics import *
from mshr import *
#from math import pi, sin, cos, sqrt
import math
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import os

In [4]:
fname = 'dMRI_data/ss-ras_data.nii.gz' 
bval_fname = 'dMRI_data/ss-ras.bval' 
bvec_fname = 'dMRI_data/ss-ras.bvec'

data, affine = load_nifti(fname)
bvals, bvecs = read_bvals_bvecs(bval_fname, bvec_fname)
print('data.shape (%d, %d, %d, %d)' % data.shape)

data_float = data.astype(np.float32)

dataMaxVal = int(max(data.flatten()))
dataMinVal = int(min(data.flatten()))

print('b values: ', bvals)
print('b vectors: ', bvecs)

data.shape (104, 104, 72, 32)
b values:  [  0. 700. 700. 700. 700. 700. 700. 700. 700. 700. 700. 700. 700. 700.
 700. 700. 700. 700. 700. 700. 700.   0. 700. 700. 700. 700. 700. 700.
 700. 700. 700. 700.]
b vectors:  [[ 0.          0.          0.        ]
 [ 0.22293058 -0.49968055 -0.83703119]
 [-0.17871778 -0.49333411 -0.85128242]
 [-0.37542525 -0.14225766 -0.91587037]
 [ 0.41157657 -0.72193778 -0.556247  ]
 [ 0.19853516 -0.93616748 -0.29012811]
 [ 0.84751403 -0.52907634 -0.04240569]
 [ 0.73912424 -0.51868695 -0.42972025]
 [ 0.43850514 -0.15385152 -0.88546205]
 [ 0.75664991 -0.17243032 -0.63067305]
 [ 0.62896943 -0.74815804  0.21132247]
 [ 0.30347744 -0.94898587  0.0855993 ]
 [ 0.28489485 -0.54959691  0.78535217]
 [ 0.62077832 -0.54730117  0.56133366]
 [ 0.97062433 -0.19699152  0.13814011]
 [ 0.83273727 -0.20222756  0.51541471]
 [-0.03439669 -0.75075191  0.65968806]
 [-0.02572841 -0.94997877  0.31125325]
 [-0.68088585 -0.5256443   0.50999278]
 [-0.36761704 -0.53581387  0.76010603]
 [-

In [5]:
center = Point(0.0, 0.0, 0.0)
sphere = Sphere(center,1.0)
mesh = generate_mesh(sphere,20) # 20 specify the resolution of the mesh
#plot(mesh)

In [6]:
# Mesh and function space
#sphere = Sphere(Point(0.0, 0.0, 0.0), 1.0)
#vmesh = generate_mesh(sphere, 5)
#mesh = BoundaryMesh(vmesh, "exterior")
#plot(mesh)

In [7]:
print(mesh.topology().dim())

3


In [8]:
# Define DC boundary condition
#tol = 0.03
#def boundary1(x, on_boundary):
#    if abs(x[0]-bvecs[1][0]) < tol and abs(x[1]-bvecs[1][1]) < tol and abs(x[2]-bvecs[1][2]) < tol:
#        print("b vector 1: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    elif abs(x[0]+bvecs[1][0]) < tol and abs(x[1]+bvecs[1][1]) < tol and abs(x[2]+bvecs[1][2]) < tol:
#        print("b vector 1: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    return (abs(x[0]-bvecs[1][0]) < tol and abs(x[1]-bvecs[1][1]) < tol and abs(x[2]-bvecs[1][2]) < tol) or (abs(x[0]+bvecs[1][0]) < tol and abs(x[1]+bvecs[1][1]) < tol and abs(x[2]+bvecs[1][2]) < tol)

#def boundary14(x, on_boundary):
#    if abs(x[0]-bvecs[14][0]) < tol and abs(x[1]-bvecs[14][1]) < tol and abs(x[2]-bvecs[14][2]) < tol:
#        print("b vector 14: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    elif abs(x[0]+bvecs[14][0]) < tol and abs(x[1]+bvecs[14][1]) < tol and abs(x[2]+bvecs[14][2]) < tol:
#        print("b vector 14: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    return (abs(x[0]-bvecs[14][0]) < tol and abs(x[1]-bvecs[14][1]) < tol and abs(x[2]-bvecs[14][2]) < tol) or (abs(x[0]+bvecs[14][0]) < tol and abs(x[1]+bvecs[14][1]) < tol and abs(x[2]+bvecs[14][2]) < tol)

#def boundary17(x, on_boundary):
#    if abs(x[0]-bvecs[17][0]) < tol and abs(x[1]-bvecs[17][1]) < tol and abs(x[2]-bvecs[17][2]) < tol:
#        print("b vector 17: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    elif abs(x[0]+bvecs[17][0]) < tol and abs(x[1]+bvecs[17][1]) < tol and abs(x[2]+bvecs[17][2]) < tol:
#        print("b vector 17: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    return (abs(x[0]-bvecs[17][0]) < tol and abs(x[1]-bvecs[17][1]) < tol and abs(x[2]-bvecs[17][2]) < tol) or (abs(x[0]+bvecs[17][0]) < tol and abs(x[1]+bvecs[17][1]) < tol and abs(x[2]+bvecs[17][2]) < tol)

#def boundary20(x, on_boundary):
#    if abs(x[0]-bvecs[20][0]) < tol and abs(x[1]-bvecs[20][1]) < tol and abs(x[2]-bvecs[20][2]) < tol:
#        print("b vector 20: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    elif abs(x[0]+bvecs[20][0]) < tol and abs(x[1]+bvecs[20][1]) < tol and abs(x[2]+bvecs[20][2]) < tol:
#        print("b vector 20: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    return (abs(x[0]-bvecs[20][0]) < tol and abs(x[1]-bvecs[20][1]) < tol and abs(x[2]-bvecs[20][2]) < tol) or (abs(x[0]+bvecs[20][0]) < tol and abs(x[1]+bvecs[20][1]) < tol and abs(x[2]+bvecs[20][2]) < tol)

#def boundary22(x, on_boundary):
#    if abs(x[0]-bvecs[22][0]) < tol and abs(x[1]-bvecs[22][1]) < tol and abs(x[2]-bvecs[22][2]) < tol:
#        print("b vector 22: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    elif abs(x[0]+bvecs[22][0]) < tol and abs(x[1]+bvecs[22][1]) < tol and abs(x[2]+bvecs[22][2]) < tol:
#        print("b vector 22: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    return (abs(x[0]-bvecs[22][0]) < tol and abs(x[1]-bvecs[22][1]) < tol and abs(x[2]-bvecs[22][2]) < tol) or (abs(x[0]+bvecs[22][0]) < tol and abs(x[1]+bvecs[22][1]) < tol and abs(x[2]+bvecs[22][2]) < tol)

#def boundary29(x, on_boundary):
#    if abs(x[0]-bvecs[29][0]) < tol and abs(x[1]-bvecs[29][1]) < tol and abs(x[2]-bvecs[29][2]) < tol:
#        print("b vector 29: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    elif abs(x[0]+bvecs[29][0]) < tol and abs(x[1]+bvecs[29][1]) < tol and abs(x[2]+bvecs[29][2]) < tol:
#        print("b vector 29: ", x[0], x[1], x[2], sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))
#    return (abs(x[0]-bvecs[29][0]) < tol and abs(x[1]-bvecs[29][1]) < tol and abs(x[2]-bvecs[29][2]) < tol) or (abs(x[0]+bvecs[29][0]) < tol and abs(x[1]+bvecs[29][1]) < tol and abs(x[2]+bvecs[29][2]) < tol)

In [9]:
V = FunctionSpace(mesh, "CG", 1)
#V = FunctionSpace(mesh, "P", 1)

#bc1 = DirichletBC(V, Constant(data_float[63,54,30,1]), boundary1)
#bc22 = DirichletBC(V, Constant(90.0), boundary22)
#bc14 = DirichletBC(V, Constant(10.0), boundary14)
#bc29 = DirichletBC(V, Constant(160.0), boundary29)
#bc17 = DirichletBC(V, Constant(200.0), boundary17)
#bc20 = DirichletBC(V, Constant(260.0), boundary20)

# Collect boundary conditions
#bcs = [DirichletBC(V, Constant(data_float[63,54,30,1]), boundary1), DirichletBC(V, Constant(data_float[63,54,30,14]), boundary14)]
#bcs = [DirichletBC(V, Constant(data_float[63,54,30,1]), boundary1, method="pointwise"), 
#       DirichletBC(V, Constant(data_float[63,54,30,14]), boundary14, method="pointwise"),
#       DirichletBC(V, Constant(data_float[63,54,30,17]), boundary17, method="pointwise"),
#       DirichletBC(V, Constant(data_float[63,54,30,20]), boundary20, method="pointwise"),
#       DirichletBC(V, Constant(data_float[63,54,30,22]), boundary22, method="pointwise"),
#       DirichletBC(V, Constant(data_float[63,54,30,29]), boundary29, method="pointwise"),]

# get coordinates of DOFs
dof_coords = V.tabulate_dof_coordinates()
# find nearest DOF:
dof1 = np.argmin(np.linalg.norm(dof_coords - bvecs[1], axis=1))
print('dof {}, x = {}'.format(dof1, dof_coords[dof1]))
dof1_sym = np.argmin(np.linalg.norm(dof_coords + bvecs[1], axis=1))
print('dof {}, x = {}'.format(dof1_sym, dof_coords[dof1_sym]))

dof14 = np.argmin(np.linalg.norm(dof_coords - bvecs[14], axis=1))
print('dof {}, x = {}'.format(dof14, dof_coords[dof14]))
dof14_sym = np.argmin(np.linalg.norm(dof_coords + bvecs[14], axis=1))
print('dof {}, x = {}'.format(dof14_sym, dof_coords[dof14_sym]))

dof17 = np.argmin(np.linalg.norm(dof_coords - bvecs[17], axis=1))
print('dof {}, x = {}'.format(dof17, dof_coords[dof17]))
dof17_sym = np.argmin(np.linalg.norm(dof_coords + bvecs[17], axis=1))
print('dof {}, x = {}'.format(dof17_sym, dof_coords[dof17_sym]))

dof20 = np.argmin(np.linalg.norm(dof_coords - bvecs[20], axis=1))
print('dof {}, x = {}'.format(dof20, dof_coords[dof20]))
dof20_sym = np.argmin(np.linalg.norm(dof_coords + bvecs[20], axis=1))
print('dof {}, x = {}'.format(dof20_sym, dof_coords[dof20_sym]))

dof22 = np.argmin(np.linalg.norm(dof_coords - bvecs[22], axis=1))
print('dof {}, x = {}'.format(dof22, dof_coords[dof22]))
dof22_sym = np.argmin(np.linalg.norm(dof_coords + bvecs[22], axis=1))
print('dof {}, x = {}'.format(dof22_sym, dof_coords[dof22_sym]))

dof29 = np.argmin(np.linalg.norm(dof_coords - bvecs[29], axis=1))
print('dof {}, x = {}'.format(dof29, dof_coords[dof29]))
dof29_sym = np.argmin(np.linalg.norm(dof_coords + bvecs[29], axis=1))
print('dof {}, x = {}'.format(dof29_sym, dof_coords[dof29_sym]))

# now define a DirichletBC at that point
bcs = [DirichletBC(V, Constant(data_float[63,54,30,1]),
                 'near(x[0], {x}) && near(x[1], {y}) && near(x[2], {z})'.format(x=dof_coords[dof1][0], y=dof_coords[dof1][1], z=dof_coords[dof1][2]),
                 'pointwise'),
      DirichletBC(V, Constant(data_float[63,54,30,1]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof1_sym][0], y=dof_coords[dof1_sym][1], z=dof_coords[dof1_sym][2]),
                 'pointwise'),
      
      DirichletBC(V, Constant(data_float[63,54,30,14]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof14][0], y=dof_coords[dof14][1], z=dof_coords[dof14][2]),
                 'pointwise'),
      DirichletBC(V, Constant(data_float[63,54,30,14]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof14_sym][0], y=dof_coords[dof14_sym][1], z=dof_coords[dof14_sym][2]),
                 'pointwise'),
      
      DirichletBC(V, Constant(data_float[63,54,30,17]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof17][0], y=dof_coords[dof17][1], z=dof_coords[dof17][2]),
                 'pointwise'),
      DirichletBC(V, Constant(data_float[63,54,30,17]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof17_sym][0], y=dof_coords[dof17_sym][1], z=dof_coords[dof17_sym][2]),
                 'pointwise'),
      
      DirichletBC(V, Constant(data_float[63,54,30,20]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof20][0], y=dof_coords[dof20][1], z=dof_coords[dof20][2]),
                 'pointwise'),
      DirichletBC(V, Constant(data_float[63,54,30,20]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof20_sym][0], y=dof_coords[dof20_sym][1], z=dof_coords[dof20_sym][2]),
                 'pointwise'),
      
      DirichletBC(V, Constant(data_float[63,54,30,22]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof22][0], y=dof_coords[dof22][1], z=dof_coords[dof22][2]),
                 'pointwise'),
      DirichletBC(V, Constant(data_float[63,54,30,22]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof22_sym][0], y=dof_coords[dof22_sym][1], z=dof_coords[dof22_sym][2]),
                 'pointwise'),
      
      DirichletBC(V, Constant(data_float[63,54,30,29]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof29][0], y=dof_coords[dof29][1], z=dof_coords[dof29][2]),
                 'pointwise'),
      DirichletBC(V, Constant(data_float[63,54,30,29]),
                 'near(x[0], {x}) && near(x[1], {y})'.format(x=dof_coords[dof29_sym][0], y=dof_coords[dof29_sym][1], z=dof_coords[dof29_sym][2]),
                 'pointwise'),
     ]

#Test BC
#u_D = Expression('x[0]*x[0] + x[1]*x[1] - 2*x[2]*x[2]', degree=2)
#bcs = [DirichletBC(V, u_D, boundary1, method="pointwise"), 
#       DirichletBC(V, u_D, boundary14, method="pointwise"),
#       DirichletBC(V, u_D, boundary17, method="pointwise"),
#       DirichletBC(V, u_D, boundary20, method="pointwise"),
#       DirichletBC(V, u_D, boundary22, method="pointwise"),
#       DirichletBC(V, u_D, boundary29, method="pointwise"),]


u = TrialFunction(V) 
v = TestFunction(V)
w = Function(V)      

#x = SpatialCoordinate(mesh)
r = Expression("x[0]", degree=1)
##theta = Expression("x[1]", degree=1)
##phi = Expression("x[2]", degree=1)
phi = Expression("x[1]", degree=1)
theta = Expression("x[2]", degree=1)


#r, phi, theta = SpatialCoordinate(mesh)
#x = r * sin(theta) * cos(phi)

#lamb = .6     
#f = Expression("lamb*(lamb + 1)*(sin(x[2]))**lamb*sin(lamb*x[1])", lamb=lamb) 
f = Constant(0.0)

#Weak formulation
#a = (Dx(u,0)*Dx(v,0) + (Constant(1.)/r)**2*Dx(u,1)*Dx(v,1) + Constant(1.)/(r*sin(phi))**2*Dx(u,2)*Dx(v,2))*dx
#a = (Dx(u,0)*Dx(v,0) + (Constant(1.)/r)**2*Dx(u,1)*Dx(v,1) + Constant(1.)/(r*sin(phi))**2*Dx(u,2)*Dx(v,2))*dx

#a1 = (u.dx(0)*v.dx(0))
#a2 = (u.dx(1)*v.dx(1))
#a3 = (Constant(1.)/sin(theta)**2)*(u.dx(2)*v.dx(2))
#a = (a2+a3)*dx

a1 = (u.dx(0)*v.dx(0))
a2 = ((Constant(1.)/r)**2)*(u.dx(1)*v.dx(1))
a3 = (Constant(1.)/(r*sin(theta))**2)*(u.dx(2)*v.dx(2))
#a4 = (cos(theta)/(sin(theta)*r**2))*u.dx(1)*v
#a5 = (Constant(2.)/r)*u.dx(0)*v
#a = (a1+a2+a3-a4-a5)*dx
a = (a1+a2+a3)*dx

#Different weak formulation
## Defining spherical coordinates
#r = Expression("sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2])", degree=2)
#theta = Expression("atan2(x[1],x[0])", degree=2)
#phi = Expression("acos(x[2]/sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))", degree=2)
#a1 = (u.dx(0)*v.dx(0))
#a2 = ((Constant(1.)/r)**2)*(u.dx(1)*v.dx(1))
#a3 = (Constant(1.)/(r*sin(theta))**2)*(u.dx(2)*v.dx(2))
#a4 = (cos(theta)/(sin(theta)*r**2))*u.dx(1)*v
#a5 = (Constant(2.)/r)*u.dx(0)*v
#a = (a1+a2+a3-a4-a5)*dx

#Different weak formulation for shpere surface
## Defining spherical coordinates
#r = Expression("sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2])", degree=2)
#phi = Expression("atan2(x[1],x[0])", degree=2)
#theta = Expression("acos(x[2]/sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))", degree=2)
#theta = Expression("atan2(x[1],x[0])", degree=2)
#phi = Expression("acos(x[2]/sqrt(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))", degree=2)
#a2 = u.dx(1)*v.dx(1)
#a3 = (Constant(1.)/sin(theta)**2)*(u.dx(2)*v.dx(2))
#a4 = (cos(theta)/sin(theta))*u.dx(1)*v
#a = (a2+a3-a4)*dx

#Different weak formulation
#a1 = (Constant(1.0)+((cos(theta)/sin(theta))-1)*cos(phi)**2)*(u.dx(0)*v.dx(0))
#a2 = u.dx(1)*v.dx(1)
#a3 = (cos(phi)**2+(sin(phi)**2/sin(theta)**2))*(u.dx(2)*v.dx(2))
#a4 = (cos(phi)**2*(cos(theta)/sin(theta)))*(u.dx(0)*v.dx(1))
#a5 = ((-cos(theta)/sin(theta))*cos(phi)*cos(theta)*sin(phi))*(u.dx(0)*v.dx(2))
#a6 = (cos(phi)**2*(cos(theta)/sin(theta)))*(u.dx(1)*v.dx(0))
#a7 = (cos(phi)*sin(theta)*sin(phi)-(sin(phi)/sin(theta)))*(u.dx(1)*v.dx(2))
#a8 = ((-cos(theta)/sin(theta))**2*cos(theta)*sin(phi)*cos(phi))*(u.dx(2)*v.dx(0))
#a9 = (-sin(phi)*cos(phi)*cos(theta)*(cos(theta)/sin(theta)))*(u.dx(2)*v.dx(1))
#a10 = (-cos(theta)*sin(phi)/r)*u.dx(0)*v
#a11 = ((Constant(2.)*sin(theta)**2+cos(theta))*(-sin(phi)/(r*sin(theta))))*u.dx(1)*v
#a12 = (cos(phi)*Constant(-2.)/r)*u.dx(2)*v
#a = (a1+a2+a3+a4+a5+a6+a7+a8+a9+a10+a11+a12)*dx


#Different weak formulation
#a1 = u.dx(0)*v.dx(0)
#a2 = u.dx(1)*v.dx(1)
#a3 = u.dx(2)*v.dx(2)
#a4 = (cos(phi)**2*(cos(theta)/sin(theta)))*(u.dx(0)*v.dx(1))
#a5 = ((-cos(theta)/sin(theta))*cos(phi)*cos(theta)*sin(phi))*(u.dx(0)*v.dx(2))
#a6 = (cos(phi)**2*(cos(theta)/sin(theta)))*(u.dx(1)*v.dx(0))
#a7 = (cos(phi)*sin(theta)*sin(phi)-(sin(phi)/sin(theta)))*(u.dx(1)*v.dx(2))
#a8 = ((-cos(theta)/sin(theta))**2*cos(theta)*sin(phi)*cos(phi))*(u.dx(2)*v.dx(0))
#a9 = (-sin(phi)*cos(phi)*cos(theta)*(cos(theta)/sin(theta)))*(u.dx(2)*v.dx(1))
#a10 = (-cos(theta)*sin(phi)/r)*u.dx(0)*v
#a11 = ((Constant(2.)*sin(theta)**2+cos(theta))*(-sin(phi)/(r*sin(theta))))*u.dx(1)*v
#a12 = (cos(phi)*Constant(-2.)/r)*u.dx(2)*v
#a = (a1+a2+a3+a10+a11+a12)*dx

#L = inner(f,v)*dx
L = f*v*dx
#L = Constant(0.0)

#for bc in bcs: 
#    bc.apply(a,L)
    
#solve(a == L, w, bc)
solve(a == L, w, bcs)

dof 284, x = [ 0.20533867 -0.5299292  -0.81404082]
dof 3360, x = [-0.19884546  0.51869316  0.82285868]
dof 959, x = [ 0.96042778 -0.16294616  0.20574202]
dof 2956, x = [-0.97159422  0.18850199 -0.11719928]
dof 2976, x = [-0.05390205 -0.93490914  0.33820986]
dof 1612, x = [ 0.02515696  0.94951011 -0.29962144]
dof 470, x = [-0.13328136  0.19307476 -0.96882616]
dof 3395, x = [ 0.12136166 -0.19656886  0.96956484]
dof 3415, x = [-0.26342136 -0.24956037  0.92674458]
dof 149, x = [ 0.29595637  0.1833843  -0.93292247]
dof 2941, x = [-0.85409494 -0.4954558  -0.1207108 ]
dof 1686, x = [0.87312859 0.46974883 0.08805139]


In [10]:
# Save solution to file in VTK format
vtkfile = File('LH/solution.pvd')
vtkfile << w

In [11]:
array_u = nodal_values_u = w.vector()
vertex_values_u = w.compute_vertex_values()

In [12]:
print(array_u)

In [13]:
for i in range(len(array_u)):
    print(array_u[i])

2.512172633631989
3.427920384545472
2.27799673152338
2.747636211264675
3.6033879978294694
1.5274466363033623
1.6741070907318207
2.7642352806560697
2.3669497904122716
5.8198871342653025
6.589270557272331
3.1958419024966935
5.585770911321019
4.11067464202828
4.5969603395117655
4.403875994574654
3.498034926165677
2.8376407508175228
1.8079638377846614
2.7102311009276505
1.9835751555558319
1.4065843763269763
0.8607423051200291
0.7083882159336412
0.25331897017784916
1.3606315649048302
1.772234509877707
2.933904625440484
2.410570729253841
2.462611554619937
5.955603134293894
3.198644539490008
3.3785033604962167
3.075956572582376
2.941371633380671
1.8871162960895573
1.4194388882910438
2.2340444715093164
0.831290226842282
-0.012547585474074273
-0.802724666398672
0.8593040317818165
1.8513335552741843
1.8133962498060887
2.6430656938658403
2.321232375683617
2.448874942343883
3.6910565605997063
1.5222914677849129
3.987111745300552
7.287720766222728
6.369703912733824
4.549841157649043
2.2041312264483

13.186832514280471
16.59491841271231
18.580293480445796
15.180559887574711
14.021751748981572
8.43641526805122
6.681101875477339
11.770026569440187
15.797904126969433
12.673903477613456
15.342113382376715
9.345264065859308
9.768366174565406
14.330852784742843
16.146051787355088
9.73770564977778
27.98528538775662
37.394946527735165
48.0
28.134613695616768
2.5641877460753655
3.6639633989956786
2.1739892950470687
3.0815924005001007
4.872329562521571
3.223241194998617
3.049766508892943
2.9758073392252453
2.862840098394557
3.0904138021587633
5.011146552085911
3.0181870803265958
6.17823536153254
9.347837379820461
19.748298825174654
10.78201340890215
1.4285150762186662
1.1647390370946522
1.9773633779703181
3.005369531744905
3.9403833443560603
7.985728192928532
10.781869404034005
21.71872662505291
14.423448317672017
11.009410499397152
8.498788923119655
4.093783649343298
-0.6185002521539391
-0.07282214326215819
-1.8139358096994183
-0.8917789403802159
-1.917309811534541
-2.769481500379808
-3.228

15.131406066294561
19.45973333908983
29.041483264440092
28.501821528303868
25.169952433254274
21.744959247970744
23.050276516245184
26.648479760380937
30.70459516545472
30.65848915262156
31.193954261514573
33.25010639120914
31.124828793758596
31.24958327714871
28.735164882313267
32.77204892561314
30.516930753255952
31.894706793939537
33.421764704953524
35.108070345006055
37.38607324884228
34.44783882959527
36.12070306468283
36.034115922893044
33.436259216579195
32.9667360339319
31.74660163365457
14.859815044993523
11.618401516240255
8.99251949845331
13.833370109393405
18.62041410927625
7.76103335796554
4.902228649455098
1.6894855239628281
-1.6186465096863374
-4.07930984916188
-2.2722789417278424
-5.63619692401023
-7.191932002452721
-7.232530353534046
-3.4410733647256064
0.9230434738092089
-0.7038143188463907
0.17689616534823918
-2.2043177619192686
-2.8788414307070234
-5.820530196763299
-6.69095364926688
-7.754842042901309
-6.720466364945265
-5.452612953389675
-4.102116837041181
-4.2743

26.314705217851575
25.132937194139775
25.485438738366756
21.65376903005016
21.196441829757003
21.939165134231498
20.747627853062244
1.503236013077602
3.7551135459704543
9.866173217948901
3.8760558760129125
3.0860434858183523
3.1058070624180356
3.1256393052621925
2.688136246886421
3.966859089956446
2.9276586574969197
3.848195583440917
3.1357371236344616
2.455049481152606
3.419871813204889
4.7772118064877835
5.399675913729914
8.044534380336472
10.685868067740468
5.8512560189041745
11.352754676330337
12.502308769535622
10.757426474563825
7.660751933832968
7.322942420059089
10.656179239850587
14.385168031254377
27.757258606645944
21.354914218685224
21.803774648155876
27.45728567508897
28.690529339186085
21.001655248400187
21.919949865738573
10.39018520033465
13.15537183658834
5.721122942186666
2.735664874983016
5.308541254183843
9.8263465570449
4.647490758762033
2.413680026787
1.6961458505526676
0.16109828694668613
0.6369292090644043
0.937984925041685
1.5477940810610569
0.3355724097103842


5.312755127195607
2.860567031151251
6.10372156844387
3.4829382222115743
2.0906704778458063
43.19814038601541
45.85283939866073
45.56906648133843
45.23090334363966
44.81539218275059
45.02806561649942
42.34869488437605
38.51380789678437
35.88344273486415
36.05319949419193
34.4750213619658
37.514646771547476
42.16745144383556
42.37363971543078
38.08713811557584
36.65637549943902
31.83089274775316
33.35011876279273
38.284492342975675
39.979414199650286
35.19588970807995
31.591088737987914
30.514193432383035
29.847149050768508
28.441080948252647
26.72781454411803
21.500851138783048
26.41598456876374
28.828056325483075
21.521963805974256
21.88137834693995
20.040328057716657
17.40608272497208
14.487794603499125
11.312190662372815
10.91008877671961
11.600574091022944
12.323857632867515
39.977453553871776
45.505099208078605
44.317629866210076
44.4315785191984
46.79502161074552
41.40836314158035
45.74967141876844
48.66578358911843
49.40992191851852
47.38587091431645
48.93482062650231
50.26802736

In [14]:
min(array_u)

-8.160053588676305

In [15]:
len(array_u)

3418

In [16]:
bvecs[1][0]

0.22293058

In [17]:
bvecs[1][2]

-0.83703119

In [18]:
data_float[63,54,30,1]

43.0

In [19]:
data_float[63,54,30,14]

46.0

In [20]:
data_float[63,54,30,17]

13.0

In [21]:
data_float[63,54,30,20]

48.0

In [22]:
data_float[63,54,30,22]

25.0

In [23]:
data_float[63,54,30,29]

52.0

In [24]:
#error_L2 = errornorm(u_D, w, 'L2')
#vertex_values_u_D = u_D.compute_vertex_values(mesh)
#vertex_values_u = w.compute_vertex_values(mesh)
#error_max = np.max(np.abs(vertex_values_u_D - vertex_values_u))
#print(error_max)
#print(error_L2)

In [25]:
bvecs[27]

array([-0.70739847, -0.15345097, -0.68995672])

In [26]:
x_point = Point(*[bvecs[27]])
mesh.bounding_box_tree().compute_first_entity_collision(x_point)

4294967295

In [27]:
#mesh.bounding_box_tree().compute_closest_point(x_point)

In [28]:
def project_point_onto_affine_subspace(p, V):
    """Projects a point onto an affine subspace

    p.shape = (N,)   OR (num_pts, N) for vectorization over many points / affine subspaces
    V.shape = (k, N) OR (num_pts, k, N) for vectorization

    N = dimension of ambient space
    k-1 = dimension of affine subspace

    For a single point / affine subspace:
        - p is the point in R^N to be projected onto the affine subspace
        - The affine subspace is the set of all affine combinations
        of rows, V[i,:], of the matrix V

    Vectorization:
    For many points / affine subspaces, each point is
    projected onto its corresponding affine subspace
        p[i,:] is the ith point
        V[i,:,:] is the matrix defining the ith affine subspace

    Example usage:
        import numpy as np
        spatial_dim = 3
        p1 = np.random.randn(spatial_dim)
        V1 = np.array([[2., 0., 0.],
                      [2., 1., 0.],
                      [2., 0., 1.]])
        p2 = np.random.randn(spatial_dim)
        V2 = np.array([[0., 3., 0.],
                       [1., 3., 0.],
                       [0., 3., 1.]])
        p = np.stack([p1, p2])
        V = np.stack([V1, V2])
        projected_p, affine_coordinates = project_point_onto_affine_subspace(p, V)
        projected_p1 = projected_p[0,:]
        projected_p2 = projected_p[1,:]
        print('p1=', p1)
        print('projected_p1=', projected_p1)
        print('p2=', p2)
        print('projected_p2=', projected_p2)
    """
    if len(p.shape) == 1:
        PP = p.reshape((1, p.shape[0]))
        VV = V.reshape((1, V.shape[0], V.shape[1]))
    else:
        PP = p
        VV = V
    num_pts, k, N = VV.shape

    if k == 1:
        affine_coordinates = np.ones((num_pts, 1))
    else:
        VV0 = VV[:, 0, :].reshape((num_pts, 1, N))
        dVV = VV[:, 1: :].reshape((num_pts, k-1, N)) - VV0
        dPP = PP.reshape((num_pts, 1, N)) - VV0

        if k == 2:
            PHI = np.sum(dVV * dVV, axis=-1).reshape((num_pts))
            RHS = np.sum(dVV * dPP, axis=-1).reshape((num_pts))
            cc_rest = (RHS / PHI).reshape((num_pts, 1))
        else:
            PHI = np.einsum('xiz,xjz->xij', dVV, dVV) # shape = (num_pts, k-1, k-1)
            iPHI = np.linalg.inv(PHI) # shape = (num_pts, k-1, k-1)

            RHS = np.sum(dVV * dPP, axis=-1)  # shape = (num_pts, k-1)
            cc_rest = np.einsum('pij,pj->pi', iPHI, RHS)  # shape = (num_pts, k-1)

        cc_first = (1. - np.sum(cc_rest, axis=1)).reshape((num_pts, 1))
        affine_coordinates = np.concatenate([cc_first, cc_rest], axis=1) # shape = (num_pts, k)

    PP_projected = np.einsum('pi,pij->pj', affine_coordinates, VV) # shape = (num_pts, N)

    if len(p.shape) == 1:
        PP_projected = PP_projected.reshape(-1)
        affine_coordinates = affine_coordinates.reshape(-1)
    return PP_projected, affine_coordinates

def powerset(s):
    # NOT MY CODE. FROM USER "hughdbrown" ON STACKOVERFLOW HERE:
    # https://stackoverflow.com/a/1482320/484944
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]

def closest_point_on_simplex(p, V):
    """Projects a point onto a simplex (triangle, tetrahedron, etc)

    p.shape = (N,)   OR (num_pts, N) for vectorization over many points/simplices
    V.shape = (k, N) OR (num_pts, k, N) for vectorization

    N = dimension of ambient space
    k-1 = dimension of simplex

    For a single point/simplex:
        - p is the point in R^N to be projected onto the simplex
        - The simplex is the set of all convex combinations
        of rows, V[i,:], of the matrix V

    Vectorization:
    For many points/simplices, each point is
    projected onto its corresponding simplex
        p[i,:] is the ith point
        V[i,:,:] is the matrix defining the ith simplex

    Example usage:
        import numpy as np
        import matplotlib.pyplot as plt
        p1 = np.array([1.1, 0.4])
        V1 = np.array([[0., 0.],
                       [0., 1.],
                       [1., 0.]])
        p2 = np.array([-0.3, 1.1])
        V2 = np.array([[-1.0, 0.],
                       [0.,   0.],
                       [-0.5, 0.5]])
        p = np.stack([p1, p2])
        V = np.stack([V1, V2])
        projected_p = closest_point_on_simplex(p, V)
        projected_p1 = projected_p[0,:]
        projected_p2 = projected_p[1,:]
        plt.figure()
        t1 = plt.Polygon(V1)
        plt.gca().add_patch(t1)
        plt.plot([p1[0], projected_p1[0]],
                 [p1[1], projected_p1[1]], 'r')
        t2 = plt.Polygon(V2)
        plt.gca().add_patch(t2)
        plt.plot([p2[0], projected_p2[0]],
                 [p2[1], projected_p2[1]], 'r')
        plt.gca().set_aspect('equal')
        plt.show()
    """
    if len(p.shape) == 1:
        PP = p.reshape((1, p.shape[0]))
        VV = V.reshape((1, V.shape[0], V.shape[1]))
    else:
        PP = p
        VV = V
    num_pts, k, N = VV.shape

    subsets = list(powerset(list(range(k)))) # e.g., [[], [0], [1], [2], [0,1], [0,2], [1,2], [0,1,2]]
    QQ = list()
    CC = list()
    for s in subsets:
        if s:
            simplicial_facet = VV[:,s,:]
            Q, C = project_point_onto_affine_subspace(PP, simplicial_facet)
            QQ.append(Q)
            CC.append(C)

    distances = np.stack([np.linalg.norm(Q - PP, axis=-1) for Q in QQ]) # shape=(num_facets, num_pts)

    good_inds = np.stack([(np.all(0. <= C, axis=1) & np.all(C <= 1., axis=1)) for C in CC]) # shape=(num_facets, num_pts)
    bad_inds = np.logical_not(good_inds)
    distances[bad_inds] = np.inf
    closest_inds = np.expand_dims(np.argmin(distances[:, :, None], axis=0), axis=0)

    QQ_stack = np.stack(QQ) # shape=(num_facets, num_pts, N)
    PP_projected = np.take_along_axis(QQ_stack, closest_inds, axis=0)[0,:,:]

    if len(p.shape) == 1:
        PP_projected = PP_projected.reshape(-1)
    return PP_projected

def closest_point_in_mesh(p, mesh):
    """Finds the nearest point in a mesh to a given point
    p is the point or points (numpy array)
    mesh is the fenics/dolfin mesh

    p.shape = (N,) OR (num_pts, N) for vectorization over many points
    N = dimension of ambient space

    Example usage:
        import numpy as np
        import dolfin as dl
        import matplotlib.pyplot as plt
        mesh = dl.UnitSquareMesh(13,9)
        num_pts = 20
        p = np.random.randn(num_pts, 2) + np.array([0.5, 0.5])
        closest_p = closest_point_in_mesh(p, mesh)
        plt.figure()
        dl.plot(mesh)
        for ii in range(num_pts):
            plt.plot([p[ii,0], closest_p[ii,0]], [p[ii,1], closest_p[ii,1]], 'b')
            plt.plot(p[ii,0], p[ii,1], '*k')
            plt.plot(closest_p[ii,0], closest_p[ii,1], '.r')
        plt.show()
    """
    if len(p.shape) == 1:
        PP = p[None,:]
    else:
        PP = p
    num_pts, N = PP.shape
    tdim = mesh.topology().dim()
    k = tdim + 1

    VV = np.zeros((num_pts, k, N))
    bbt = mesh.bounding_box_tree()
    for ii in range(num_pts):
        pi = PP[ii,:]
        closest_entity, closest_distance = bbt.compute_closest_entity(Point(pi))
        closest_cell = mesh.cells()[closest_entity]
        vertices_of_closest_cell = mesh.coordinates()[closest_cell, :]
        VV[ii, :, :] = vertices_of_closest_cell

    closest_PP = closest_point_on_simplex(PP, VV)

    if len(p.shape) == 1:
        closest_PP = closest_PP.reshape(-1)
    return closest_PP

In [29]:
closest_p = closest_point_in_mesh(bvecs[27], mesh)
print(bvecs[27])
print(closest_p)
print(w(closest_p))

[-0.70739847 -0.15345097 -0.68995672]
[-0.54275357 -0.45549641 -0.64830289]
36.31635554313265


In [30]:
closest_p = closest_point_in_mesh(-bvecs[27], mesh)
print(-bvecs[27])
print(closest_p)
print(w(closest_p))

[0.70739847 0.15345097 0.68995672]
[0.70147975 0.15204245 0.68353812]
8.0791937388216


In [31]:
array_1 = np.array([[1, 2, 7], [3, 4, 8]])

In [32]:
array_2 = np.array([[1, 2], [3, 9], [4, 16]]) 

In [33]:
np.dot(array_1, array_2)

array([[ 35, 132],
       [ 47, 170]])